In [ ]:
#Loading in Packages and Data

#Importing Packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr
import os; import time
import pickle
import h5py
###############################################################
def coefs(coefficients,degree):
    coef=coefficients
    coefs=""
    for n in range(degree, -1, -1):
        string=f"({coefficients[len(coef)-(n+1)]:.1e})"
        coefs+=string + f"x^{n}"
        if n != 0:
            coefs+=" + "
    return coefs
###############################################################

# Importing Model Data
check=False
dir='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
###############################################################

# # dx = 1 km; Np = 1M; Nt = 5 min
# data=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_1km_5min.nc') #***
# parcel=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_pdata_1km_5min_1e6.nc') #***
# res='1km';t_res='5min'
# Np_str='1e6'

# dx = 1km; Np = 50M
#Importing Model Data
check=False
dir2='/home/air673/koa_scratch/'
data=xr.open_dataset(dir2+'cm1out_1km_1min.nc') #***
parcel=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_50M.nc') #***
res='1km'; t_res='1min'; Np_str='50e6'

# # dx = 1km; Np = 100M
# #Importing Model Data
# check=False
# dir2='/home/air673/koa_scratch/'
# data=xr.open_dataset(dir2+'cm1out_1km_1min.nc') #***
# parcel=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_100M.nc') #***
# res='1km'; t_res='1min'; Np_str='100e6'

# #uncomment if using 250m data
# #Importing Model Data
# check=False
# dir2='/home/air673/koa_scratch/'
# data=xr.open_dataset(dir2+'cm1out_250m.nc') #***
# # # parcel=xr.open_dataset(dir2+'cm1out_pdata_250m.nc') #***

# # Restricts the timesteps of the data from timesteps0 to 140
# data=data.isel(time=np.arange(0,400+1))
# # # parcel=parcel.isel(time=np.arange(0,400+1))
# res='250m'

In [ ]:
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
path=dir2+'../Functions/'
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions


# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

In [ ]:
#JOB ARRAY SETUP
job_array=False;index_adjust=0
job_array=True

if job_array==True:

    num_jobs=300 #how many total jobs are being run? i.e. array=1-100 ==> num_jobs=100 #***
    total_elements=len(data['time']) #total num of variables

    if num_jobs >= total_elements:
        raise ValueError("Number of jobs cannot be greater than or equal to total elements.")
    
    job_range = total_elements // num_jobs  # Base size for each chunk
    remaining = total_elements % num_jobs   # Number of chunks with 1 extra 
    
    # Function to compute the start and end for each job_id
    def get_job_range(job_id, num_jobs):
        job_id-=1
        # Add one extra element to the first 'remaining' chunks
        start_job = job_id * job_range + min(job_id, remaining)
        end_job = start_job + job_range + (1 if job_id < remaining else 0)
    
        if job_id == num_jobs - 1: 
            end_job = total_elements #- 1
        return start_job, end_job
    # def job_testing(num_jobs):
    #     #TESTING
    #     start=[];end=[]
    #     for i,job_id in enumerate(range(1,num_jobs+1)):
    #         start_job, end_job = get_job_range(job_id,num_jobs)
    #         print(start_job,end_job)
    #         start.append(start_job)
    #         end.append(end_job)
    #         # plt.scatter(i,end_job-start_job)
    #     print(np.all(start!=end))
    #     print(len(np.unique(start))==len(start))
    #     print(len(np.unique(end))==len(end))
    # job_testing(num_jobs)
    
    job_id = int(os.environ.get('SLURM_ARRAY_TASK_ID', 0)) #this is the current SBATCH job id
    if job_id==0: job_id=1
    start_job, end_job = get_job_range(job_id, num_jobs)
    index_adjust=start_job
    print(f'start_job = {start_job}, end_job = {end_job}')

In [ ]:
if job_array==True:
    #Indexing Array with JobArray
    data=data.isel(time=slice(start_job,end_job))
    parcel=parcel.isel(time=slice(start_job,end_job))
    #(for 150_000_000 parcels use 500-1000 jobs)

In [ ]:
# Reading Back Data Later
##############

def make_data_dict(in_file,var_names,read_type):
    if read_type=='h5py':
        with h5py.File(in_file, 'r') as f:
            if job_array==True:
                data_dict = {var_name: f[var_name][start_job:end_job] for var_name in var_names}
            elif job_array==False:
                data_dict = {var_name: f[var_name][:] for var_name in var_names}
            
    elif read_type=='xarray':
        in_data = xr.open_dataset(
            in_file,
            engine='h5netcdf',
            phony_dims='sort',
            chunks={'phony_dim_0': 100, 'phony_dim_1': 100_000} 
        )
        if job_array==True:
            data_dict = {k: in_data[k][start_job:end_job].compute().data for k in var_names}
        elif job_array==False:
            data_dict = {k: in_data[k][:].compute().data for k in var_names}
    return data_dict

# read_type='xarray'
read_type='h5py'

In [ ]:
import h5py
dir2=dir+'Project_Algorithms/Lagrangian_Arrays/'
in_file=dir2+f'lagrangian_binary_array_{res}_{t_res}_{Np_str}.h5'

var_names = ['A_g', 'A_c', 'W', 'QCQI', 'Z', 'Y', 'X']
data_dict = make_data_dict(in_file,var_names,read_type)
A_g, A_c, W, QCQI, Z, Y, X = (data_dict[k] for k in var_names)

# #Making Time Matrix
# rows, cols = A.shape[0], A.shape[1]
# T = np.arange(rows).reshape(-1, 1) * np.ones((1, cols), dtype=int)
check_memory(globals())

In [ ]:
# Reading Back Data Later
##############
import h5py
dir2=dir+'Project_Algorithms/Lagrangian_Arrays/'
in_file=dir2+f'VARS_binary_array_{res}_{t_res}_{Np_str}.h5'

var_names = ['QV','TH','TH_E','BUOYANCY','HMC']
data_dict = make_data_dict(in_file,var_names,read_type)
QV, TH, TH_E, BUOYANCY, HMC = (data_dict[k] for k in var_names)
check_memory(globals())

In [ ]:
#DOMAIN SUBSETTING
############################################################
ocean_percent=2/8

left_to_coast=data['xh'][0]+(data['xh'][-1]-data['xh'][0])*ocean_percent
where_coast_xh=np.where(data['xh']>=left_to_coast)[0][0]#-25
where_coast_xf=np.where(data['xf']>=left_to_coast)[0][0]#-25
end_xh=len(data['xh'])-1-50
end_xf=len(data['xf'])-1-50

print(f'x in {0}:{where_coast_xh-1} FOR SEA')
print(f'x in {where_coast_xh}:{end_xh} FOR LAND')
# t_end=78 
# if res=='250m':t_end=410
# print(f't in {0}:{t_end} (6.5 hours)')
if t_res=="5min":
    t_start=36
elif t_res=="1min":
    t_start=36*5
print(f't in {t_start}:end (8 hours)')

#####################################################################

#SUBSETTING CODE
A_g[(X<where_coast_xh)|(X>end_xh)]=0
A_c[(X<where_coast_xh)|(X>end_xh)]=0

#SUBSETTING TIME FOR JOB ARRAY
if job_array==True:
    if end_job<=t_start:
        A_g[0:t_start]=0
        A_c[0:t_start]=0
elif job_array==False:
    A_g[0:t_start]=0
    A_c[0:t_start]=0

In [ ]:
##########################################################################################
#Functions
# Full Profile function makes profile together for all timesteps. AveragedProfiles funciton takes the final mean of the combined profile.

In [ ]:
dir2=dir+'Project_Algorithms/Domain_Profiles/'

def LagrangianProfiles(type):
    print(f'currently working on type {type}')
    
    #Create Storage Arrays for All Variables
    vars=['w','qv','qc_plus_qi','th','th_e','buoyancy','HMC']
    for var in vars:
        zhs=data['zh'].values
        globals()[f"profile_{var}"]=np.zeros((len(zhs), 3)) #column 1: var, column 2: counter, column 3: list of zhs
        globals()[f"profile_{var}"][:,2]=zhs
    
    Nt=len(data['time'])
    for t in np.arange(Nt):
        # if np.mod(t,20)==0: print(f"time {t}")
        
        #Call Variables and Store in a Dictionary
        var_data_map = {
            'w': W,
            'qv': QV,
            'qc_plus_qi': QCQI,
            'th': TH,
            'th_e': TH_E,
            'buoyancy': BUOYANCY,
            'HMC': HMC
            }
    
        #Get the Lagrangian Parcel Grid Locations
        if type=='general':
            where_g=np.where(A_g[t,:]==1)
            zs=Z[t,where_g[0]]
        elif type=='cloudy':
            where_c=np.where(A_c[t,:]==1)
            zs=Z[t,where_c[0]]
    
        #For Each Variable, Link profile_array to profile_{var} and Add Profile Elements to Storage Array
        for var in vars:
            profile_array = globals()[f"profile_{var}"]
            if type=='general':
                var_data = var_data_map[var][t,where_g[0]]
            elif type=='cloudy':
                var_data = var_data_map[var][t,where_c[0]]
            
            np.add.at(profile_array[:, 0], zs, var_data)
            np.add.at(profile_array[:, 1], zs, 1)
    
    #OUTPUTTING RESULTS       
    if type=='general':
        output_file=dir2+f'job_out/general_lagrangian_profiles_{res}_{t_res}_{Np_str}'
    elif type=='cloudy':
        output_file=dir2+f'job_out/cloudy_lagrangian_profiles_{res}_{t_res}_{Np_str}'
    if job_array==True:
        output_file+=f'_{job_id}.h5'
    elif job_array==False:
        output_file+=f'.h5'

    print(f"saving as {output_file}")
    import h5py
    with h5py.File(output_file, 'w') as f:
        f.create_dataset('profile_w', data=profile_w, compression="gzip")
        f.create_dataset('profile_qv', data=profile_qv, compression="gzip")
        f.create_dataset('profile_qc_plus_qi', data=profile_qc_plus_qi, compression="gzip")
        f.create_dataset('profile_th', data=profile_th, compression="gzip")
        f.create_dataset('profile_th_e', data=profile_th_e, compression="gzip")
        f.create_dataset('profile_buoyancy', data=profile_buoyancy, compression="gzip")
        f.create_dataset('profile_HMC', data=profile_HMC, compression="gzip")
    print('done\n')
    return profile_w,profile_qv,profile_qc_plus_qi,profile_th,profile_th_e,profile_buoyancy,profile_HMC

In [ ]:
#RUNNING
[profile_w,profile_qv,profile_qc_plus_qi,profile_th,profile_th_e,profile_buoyancy,profile_HMC]=LagrangianProfiles(type='general')

[profile_w,profile_qv,profile_qc_plus_qi,profile_th,profile_th_e,profile_buoyancy,profile_HMC]=LagrangianProfiles(type='cloudy')

In [ ]:
#########################################
#RECOMBINE SEPERATE JOB_ARRAYS AFTER
recombine=False #KEEP FALSE WHEN JOB ARRAY IS RUNNING
# recombine=True

In [ ]:
if recombine==True:
    dir2=dir+'Project_Algorithms/Domain_Profiles/'
    
    types=['general','cloudy']
    for type in types:
        #MAKING OUTPUT FILE PATH
        if type == "general":
            output_file = dir2+f'job_out/general_lagrangian_profiles_{res}_{t_res}_{Np_str}.h5' 
        elif type == "cloudy":
            output_file = dir2+f'job_out/cloudy_lagrangian_profiles_{res}_{t_res}_{Np_str}.h5'
        
        #MAKING PROFILES DICTIONARY
        zhs = data['zh'].values
        profiles = {}  # Store profiles for all variables
        vars_list = ['w', 'qv', 'qc_plus_qi', 'th', 'th_e', 'buoyancy', 'HMC']
        for var in vars_list:
            profiles[var] = np.zeros((len(zhs), 3))  # column 1: var, column 2: counter, column 3: list of zhs
            profiles[var][:, 2] = zhs 
        
        num_jobs=300
        for job_id in np.arange(1,num_jobs+1):
            if np.mod(job_id,10)==0: print(f"job_id = {job_id}")
    
            #CALLING IN DATA
            if type == "general":
                input_file = dir2+f'job_out/general_lagrangian_profiles_{res}_{t_res}_{Np_str}_{job_id}.h5' 
            elif type == "cloudy":
                input_file = dir2+f'job_out/cloudy_lagrangian_profiles_{res}_{t_res}_{Np_str}_{job_id}.h5'
    
            #COMPILING PROFILES
            with h5py.File(input_file, 'r') as f:
                for var in vars_list:  
                    profiles[var][:,0:1+1]+=f[f'profile_{var}'][:,0:1+1]
        
        #SAVING INTO FINAL FORM
        with h5py.File(output_file, 'w') as f:
            for var in profiles:
                profile_var = profiles[var]
                f.create_dataset(f'profile_{var}', data=profile_var, compression="gzip")